<a href="https://colab.research.google.com/github/AryanSai/Deep-Learning/blob/main/23352_Asmt_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn #building blocks needed to build the NN
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#loading the training data
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
#loading the test data
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

#wraping iterators around the datasets
train_dataloader=DataLoader(training_data,batch_size=32)
test_dataloader=DataLoader(test_data,batch_size=32)

100%|██████████| 26421880/26421880 [00:01<00:00, 13482239.97it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 228467.49it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 4214982.87it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3674030.46it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
class NeuralNetwork(nn.Module): #subclass of the nn.Module
    def __init__(self): #constructor

      super().__init__() #calling the parent's constructor

      self.flatten = nn.Flatten() #flatten layer before sending to the linear layers
      #defining the three layers with ReLu activations

      self.linear_relu_stack = nn.Sequential(
          nn.Linear(28*28,64), #(input,output) ;64 hidden nodes in layer 1
          nn.ReLU(),
          nn.Linear(64,128), #128 hidden nodes in layer 1
          nn.ReLU(),
          nn.Linear(128,10),
    )

    def forward(self,x): #the forward pass of the NN
      x=self.flatten(x) #flatten the input tensor
      logits=self.linear_relu_stack(x) #pass the input through linear stack and store output in logits
      return logits

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
      size = len(dataloader.dataset) #get the size of the dataset
      model.train()
      for batch, (X,y) in enumerate(dataloader):
        #computing the prediction and loss
        pred=model(X)
        loss=loss_fn(pred,y)
        #back propagation
        loss.backward() # Compute gradients
        optimizer.step() # Update weights
        optimizer.zero_grad() # Zero out gradients for next iteration

        if batch%1000 == 0:
          loss, current = loss.item(), (batch+1)*len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval() #set the model to evaluation mode

    size = len(dataloader.dataset)
    num_batches = len(dataloader) #total number of batches in the data loader

    test_loss=0 #total loss
    correct = 0 #number of correct prediction

    with torch.no_grad(): #disables gradient computation
        for X, y in dataloader:
            pred = model(X) #forward pass to obtain predictions from the model
            test_loss += loss_fn(pred, y).item() #find loss and accumulate it
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() #count correct prediction
    test_loss /= num_batches #average test loss over all batches
    correct /= size #accuracy
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [5]:

model = NeuralNetwork() #creating an instance of the nn

learning_rate = 1e-2 #rate at which the parameters are updated
batch_size = 64 #no. of samples processed before updating the parameters
epochs = 30 #no. of times the whole dataset is iterated over

loss_fn = nn.CrossEntropyLoss() #loss function(cross-entropy)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #stochastic gradient descent

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.309417  [   32/60000]
loss: 0.746327  [32032/60000]
Test Error: 
 Accuracy: 76.3%, Avg loss: 0.662994 

Epoch 2
-------------------------------
loss: 0.562872  [   32/60000]
loss: 0.633786  [32032/60000]
Test Error: 
 Accuracy: 80.3%, Avg loss: 0.552814 

Epoch 3
-------------------------------
loss: 0.438812  [   32/60000]
loss: 0.559128  [32032/60000]
Test Error: 
 Accuracy: 81.7%, Avg loss: 0.513358 

Epoch 4
-------------------------------
loss: 0.378524  [   32/60000]
loss: 0.531105  [32032/60000]
Test Error: 
 Accuracy: 82.6%, Avg loss: 0.488615 

Epoch 5
-------------------------------
loss: 0.334169  [   32/60000]
loss: 0.527381  [32032/60000]
Test Error: 
 Accuracy: 83.2%, Avg loss: 0.469021 

Epoch 6
-------------------------------
loss: 0.309080  [   32/60000]
loss: 0.526881  [32032/60000]
Test Error: 
 Accuracy: 83.6%, Avg loss: 0.453826 

Epoch 7
-------------------------------
loss: 0.292836  [   32/60000]
loss: 0.531590  [3

Epoch 29
-------------------------------
loss: 0.198498  [   32/60000]
loss: 0.402522  [32032/60000]
Test Error:
 Accuracy: 87.6%, Avg loss: 0.353605

Epoch 30
-------------------------------
loss: 0.195382  [   32/60000]
loss: 0.398971  [32032/60000]
Test Error:
 Accuracy: 87.7%, Avg loss: 0.352234

Done!